In [ ]:
import numpy as np
import pandas as pd

# Constants
P_tx = 250  # Max transmit power in mW
P_tx_dBm = 10 * np.log10(P_tx)  # Convert to dBm
noise_figure = 5  # dB
thermal_noise_density = -174  # dBm/Hz
bandwidth = 5e6  # Bandwidth in Hz (5 MHz)

# Function to calculate path loss
def path_loss(d):
    return 128.1 + 37.6 * np.log10(d)

# Function to calculate received power
def received_power(path_loss):
    return P_tx_dBm - path_loss

# Function to calculate thermal noise power
def thermal_noise(bandwidth, noise_figure):
    return thermal_noise_density + 10 * np.log10(bandwidth) + noise_figure

# Function to calculate SNR (in dB)
def snr(received_power, thermal_noise_power):
    return received_power - thermal_noise_power

# Function to calculate data rate using Shannon Capacity formula
def data_rate(snr_dB, bandwidth):
    snr_linear = 10**(snr_dB / 10)  # Convert to linear scale
    return bandwidth * np.log2(1 + snr_linear) / 1e6  # Mbps

# Function to calculate latency (simple formula assuming 1 kb of data)
def latency(data_rate):
    packet_size = 1e3  # 1 KB packet
    propagation_delay = 0.000005  # Assuming small fixed propagation delay in seconds (5 ms)
    transmission_time = packet_size / (data_rate * 1e6)  # Transmission time in seconds
    total_latency = (transmission_time + propagation_delay) * 1000  # Convert to milliseconds
    return total_latency

# Generate data for D2D pairs and CUs
np.random.seed(42)  # For reproducibility
num_pairs = 5000  # Number of D2D pairs to simulate

# Generate distances between 10m and 500m, but skew towards smaller distances for high data rate
distances = np.random.uniform(10, 500, num_pairs)  # Random distances

# Create an empty dataframe
columns = ['D2D Pair / CU ID', 'Distance (m)', 'Path Loss (dB)', 'Received Power (dBm)',
           'SNR (dB)', 'Data Rate (Mbps)', 'Latency (ms)', 'Bandwidth (MHz)']
data = []

# Simulate data for each pair
for i, dist in enumerate(distances):
    pl = path_loss(dist)  # Calculate path loss
    rp = received_power(pl)  # Calculate received power
    noise = thermal_noise(bandwidth, noise_figure)  # Calculate thermal noise
    snr_dB = snr(rp, noise)  # Calculate SNR
    dr = data_rate(snr_dB, bandwidth)  # Calculate data rate
    lat = latency(dr)  # Calculate latency

    # Make sure the data reflects realistic trends
    if dist < 100:
        # For distances < 100m, ensure high data rate and low latency
        dr = min(dr, 50)  # Cap data rate to 50 Mbps for closer distances
        lat = min(lat, 10)  # Cap latency to 10 ms for close distances
    elif dist > 400:
        # For distances > 400m, ensure low data rate and higher latency
        dr = min(dr, 2)  # Cap data rate to 2 Mbps for long distances
        lat = max(lat, 50)  # Increase latency for larger distances

    # Store the results
    data.append([f'D2D-{i+1}', dist, pl, rp, snr_dB, dr, lat, bandwidth / 1e6])

# Convert to DataFrame
df = pd.DataFrame(data, columns=columns)

# Display the first few rows of the generated dataframe
print(df.head())



  D2D Pair / CU ID  Distance (m)  Path Loss (dB)  Received Power (dBm)  \
0            D2D-1    193.524658      214.081285           -190.101885   
1            D2D-2    475.850010      228.772875           -204.793475   
2            D2D-3    368.677031      224.605893           -200.626493   
3            D2D-4    303.342657      221.420699           -197.441299   
4            D2D-5     86.449134      200.922200           -176.942800   

     SNR (dB)  Data Rate (Mbps)  Latency (ms)  Bandwidth (MHz)  
0  -88.091585      1.119402e-08  8.933343e+07              5.0  
1 -102.783175      3.800385e-10  2.631312e+09              5.0  
2  -98.616193      9.920307e-10  1.008033e+09              5.0  
3  -95.430999      2.065592e-09  4.841228e+08              5.0  
4  -74.932500      2.316832e-07  1.000000e+01              5.0  


In [ ]:
import numpy as np
import pandas as pd

# Constants
P_tx = 250  # Max transmit power in mW
P_tx_dBm = 10 * np.log10(P_tx)  # Convert transmit power to dBm (log scale)
noise_figure = 5  # Noise figure in dB (assumed constant)
thermal_noise_density = -174  # Thermal noise density in dBm/Hz (standard value)
bandwidth = 5e6  # Bandwidth in Hz (5 MHz)

# Function to calculate path loss using the given formula
def path_loss(d):
    return 128.1 + 37.6 * np.log10(d)  # Path loss formula (log-distance model)

# Function to calculate received power using path loss
def received_power(path_loss):
    return P_tx_dBm - path_loss  # Received power in dBm (subtract path loss from transmit power)

# Function to calculate thermal noise power based on bandwidth and noise figure
def thermal_noise(bandwidth, noise_figure):
    return thermal_noise_density + 10 * np.log10(bandwidth) + noise_figure  # Total noise power in dBm

# Function to calculate Signal-to-Noise Ratio (SNR) in dB
def snr(received_power, thermal_noise_power):
    return received_power - thermal_noise_power  # SNR in dB (difference between received power and noise)

# Function to calculate data rate in Mbps using the Shannon Capacity formula
def data_rate(snr_dB, bandwidth):
    snr_linear = 10**(snr_dB / 10)  # Convert SNR from dB to linear scale
    return bandwidth * np.log2(1 + snr_linear) / 1e6  # Data rate in Mbps (Shannon formula)

# Function to calculate latency in ms (considering transmission time and fixed propagation delay)
def latency(data_rate):
    packet_size = 1e3  # Assume packet size is 1 KB (1e3 bytes)
    propagation_delay = 0.000005  # Assume small propagation delay in seconds (5 ms)
    transmission_time = packet_size / (data_rate * 1e6)  # Transmission time in seconds
    total_latency = (transmission_time + propagation_delay) * 1000  # Total latency in milliseconds
    return total_latency

# Generate data for D2D pairs and CUs
np.random.seed(42)  # Set random seed for reproducibility
num_pairs = 5000  # Total number of D2D pairs to simulate (5000 rows of data)

# Generate distances randomly between 10m and 500m
distances = np.random.uniform(10, 500, num_pairs)  # Uniform distribution of distances

# Create a list of column names and an empty list to hold the data
columns = ['D2D Pair / CU ID', 'Distance (m)', 'Path Loss (dB)', 'Received Power (dBm)',
           'SNR (dB)', 'Data Rate (Mbps)', 'Latency (ms)', 'Bandwidth (MHz)']
data = []

# Loop through each D2D pair and calculate the relevant parameters
for i, dist in enumerate(distances):
    # Step 1: Calculate path loss using the distance (log-distance model)
    pl = path_loss(dist)  # Path loss in dB

    # Step 2: Calculate the received power based on path loss (received power = transmit power - path loss)
    rp = received_power(pl)  # Received power in dBm

    # Step 3: Calculate thermal noise power based on bandwidth and noise figure
    noise = thermal_noise(bandwidth, noise_figure)  # Total thermal noise in dBm

    # Step 4: Calculate Signal-to-Noise Ratio (SNR) in dB
    snr_dB = snr(rp, noise)  # SNR in dB (difference between received power and thermal noise)

    # Step 5: Calculate the data rate in Mbps using Shannon Capacity formula
    dr = data_rate(snr_dB, bandwidth)  # Data rate in Mbps

    # Step 6: Calculate latency in milliseconds based on the data rate
    lat = latency(dr)  # Latency in ms (transmission time + fixed propagation delay)

    # Step 7: Apply logical conditions for data rate and latency based on distance
    if dist < 100:
        # For distances < 100m, assume high data rate and low latency
        dr = min(dr, 50)  # Cap data rate to 50 Mbps for closer distances
        lat = min(lat, 10)  # Cap latency to 10 ms for closer distances
    elif dist > 400:
        # For distances > 400m, assume low data rate and high latency
        dr = min(dr, 2)  # Cap data rate to 2 Mbps for longer distances
        lat = max(lat, 50)  # Increase latency to at least 50 ms for larger distances

    # Append the calculated values to the data list
    data.append([f'D2D-{i+1}', dist, pl, rp, snr_dB, dr, lat, bandwidth / 1e6])

# Convert the list of data to a DataFrame
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to an Excel file on your local device
import os
excel_file_path = r"C:\Users\Keerthi Balaji\Desktop\D2D_Resource_Allocation.xlsx"
df.to_excel(excel_file_path, index=False)

folder_path = r"C:\Users\Keerthi Balaji\OneDrive\Documents\Projects\D2D"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)  # Create folder if it doesn’t exist

excel_file_path = os.path.join(folder_path, "D2D_Resource_Allocation_Cleaned.xlsx")
df.to_excel(excel_file_path, index=False)

print(f"Saving file to: {excel_file_path}")
df.to_excel(excel_file_path, index=False)

# Print the first few rows of the DataFrame for verification
print(df.head())



Saving file to: C:\Users\Keerthi Balaji\OneDrive\Documents\Projects\D2D/D2D_Resource_Allocation_Cleaned.xlsx
  D2D Pair / CU ID  Distance (m)  Path Loss (dB)  Received Power (dBm)  \
0            D2D-1    193.524658      214.081285           -190.101885   
1            D2D-2    475.850010      228.772875           -204.793475   
2            D2D-3    368.677031      224.605893           -200.626493   
3            D2D-4    303.342657      221.420699           -197.441299   
4            D2D-5     86.449134      200.922200           -176.942800   

     SNR (dB)  Data Rate (Mbps)  Latency (ms)  Bandwidth (MHz)  
0  -88.091585      1.119402e-08  8.933343e+07              5.0  
1 -102.783175      3.800385e-10  2.631312e+09              5.0  
2  -98.616193      9.920307e-10  1.008033e+09              5.0  
3  -95.430999      2.065592e-09  4.841228e+08              5.0  
4  -74.932500      2.316832e-07  1.000000e+01              5.0  
